# constraints

Este notebook se encarga de asignar las llaves primarias y foráneas a las tablas de hechos y dimensiones

### Importación de librerías

In [31]:
import yaml
from sqlalchemy import create_engine, text
from sqlalchemy.inspection import inspect

### Definición de las llaves foráneas

In [35]:
props = {
    "hecho_novedad" : {
        "key_tipo_novedad" : {
            "column" : "key_tipo_novedad",
            "ref_table" : "dim_tipo_novedad",
            "ref_column" : "key_dim_tipo_novedad"
        },
    },
    "hecho_entrega_mensajeria_diario" : {
        "key_fecha" : {
            "column" : "key_fecha",
            "ref_table" : "dim_fecha",
            "ref_column" : "key_dim_fecha"
        },
        "key_cliente" : {
            "column" : "key_cliente",
            "ref_table" : "dim_cliente",
            "ref_column" : "key_dim_cliente"
        },
        "key_sede" : {
            "column" : "key_sede",
            "ref_table" : "dim_sede",
            "ref_column" : "key_dim_sede"
        },
    },
    "hecho_entrega_mensajeria_hora" : {
        "key_hora" : {
            "column" : "key_hora",
            "ref_table" : "dim_hora",
            "ref_column" : "key_dim_hora"
        },
        "key_mensajero" : {
            "column" : "key_mensajero",
            "ref_table" : "dim_mensajero",
            "ref_column" : "key_dim_mensajero"
        },
    },
    "hecho_entrega_mensajeria_acumulada" : {
        "key_fecha_inicio" : {
            "column" : "key_fecha_inicio",
            "ref_table" : "dim_fecha",
            "ref_column" : "key_dim_fecha"
        },
        "key_fecha_asignacion" : {
            "column" : "key_fecha_asignacion",
            "ref_table" : "dim_fecha",
            "ref_column" : "key_dim_fecha"
        },
        "key_fecha_recogida" : {
            "column" : "key_fecha_recogida",
            "ref_table" : "dim_fecha",
            "ref_column" : "key_dim_fecha"
        },
        "key_fecha_entrega" : {
            "column" : "key_fecha_entrega",
            "ref_table" : "dim_fecha",
            "ref_column" : "key_dim_fecha"
        },
        "key_fecha_cierre" : {
            "column" : "key_fecha_cierre",
            "ref_table" : "dim_fecha",
            "ref_column" : "key_dim_fecha"
        },
        "key_hora_inicio" : {
            "column" : "key_hora_inicio",
            "ref_table" : "dim_hora",
            "ref_column" : "key_dim_hora"
        },
        "key_hora_asignacion" : {
            "column" : "key_hora_asignacion",
            "ref_table" : "dim_hora",
            "ref_column" : "key_dim_hora"
        },
        "key_hora_recogida" : {
            "column" : "key_hora_recogida",
            "ref_table" : "dim_hora",
            "ref_column" : "key_dim_hora"
        },
        "key_hora_entrega" : {
            "column" : "key_hora_entrega",
            "ref_table" : "dim_hora",
            "ref_column" : "key_dim_hora"
        },
        "key_hora_cierre" : {
            "column" : "key_hora_cierre",
            "ref_table" : "dim_hora",
            "ref_column" : "key_dim_hora"
        },
    },
}

### Definición de funciones

Estas funciones se encargan de verificar y asignar las llaves primarias y foráneas

In [37]:
def define_primary_key(conn, table_name, column_name):
    try:
        query = text(f'ALTER TABLE {table_name} ADD PRIMARY KEY ({column_name})')
        conn.execute(query)
        conn.commit()
        print(f"Primary key added on {table_name} for column {column_name}.")
    except Exception as e:
        print(f"Error adding primary key on {table_name}: {e}")

def primary_key_exists(inspector, table_name):
    pk_constraint = inspector.get_pk_constraint(table_name)
    return pk_constraint is not None and len(pk_constraint.get('constrained_columns', [])) > 0

def define_foreigns_key(conn, table_name, column_name, ref_table, ref_column):
    try:
        query = text(f'ALTER TABLE {table_name} ADD FOREIGN KEY ({column_name}) REFERENCES {ref_table}({ref_column})')
        conn.execute(query)
        conn.commit()
        print(f"Foreign key added on {table_name} for column {column_name}.")
    except Exception as e:
        print(f"Error adding foreign key on {table_name}: {e}")

def foreign_key_exists(inspector, table_name, column_name):
    fk_constraints = inspector.get_foreign_keys(table_name)
    for fk in fk_constraints:
        if column_name in fk.get('constrained_columns', []):
            return True
    return False

### Asignación de llaves

In [38]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)
    configFuente = config['fuente']
    configBodega = config['bodega']

urlBodega = f"{configBodega['driver']}://{configBodega['user']}:{configBodega['password']}@{configBodega['host']}:{configBodega['port']}/{configBodega['db']}"

engineBodega = create_engine(urlBodega)

inspector = inspect(engineBodega)
tnames = inspector.get_table_names()

with engineBodega.connect() as conn: 
    for table in tnames:
        primary_key_column = f'key_{table}'
        
        columns = inspector.get_columns(table)
        column_names = [column['name'] for column in columns]

        if primary_key_exists(inspector, table):
            print(f"Table {table} already has a primary key. Skipping.")
            continue

        if primary_key_column in column_names:
            define_primary_key(conn, table, primary_key_column)
        else:
            print(f"Column {primary_key_column} does not exist in table {table}.")

    conn.close()

print("\n")

with engineBodega.connect() as conn:
    for fact_table, foreign_keys in props.items():
        print("fact_table>", fact_table)
        for foreign_key, foreign_keys_info in foreign_keys.items():
            print(">")
            print("foreign_key>", foreign_key)

            if not foreign_key_exists(inspector, fact_table, foreign_keys_info['column']):
                define_foreigns_key(conn, fact_table, foreign_keys_info['column'], foreign_keys_info['ref_table'], foreign_keys_info['ref_column'])
            else:
                print(f"Foreign key already exists on {table} for column {foreign_keys_info['column']}. Skipping.")
        print("=================")

            
    conn.close

Table dim_cliente already has a primary key. Skipping.
Table dim_fecha already has a primary key. Skipping.
Table hecho_entrega_mensajeria_diario already has a primary key. Skipping.
Table dim_hora already has a primary key. Skipping.
Column key_hecho_entrega_mensajeria_acumulada does not exist in table hecho_entrega_mensajeria_acumulada.
Table dim_mensajero already has a primary key. Skipping.
Table dim_sede already has a primary key. Skipping.
Table hecho_entrega_mensajeria_hora already has a primary key. Skipping.
Table dim_tipo_novedad already has a primary key. Skipping.
Table hecho_novedad already has a primary key. Skipping.


fact_table> hecho_novedad
>
foreign_key> key_tipo_novedad
Foreign key added on hecho_novedad for column key_tipo_novedad.
fact_table> hecho_entrega_mensajeria_diario
>
foreign_key> key_fecha
Foreign key added on hecho_entrega_mensajeria_diario for column key_fecha.
>
foreign_key> key_cliente
Foreign key added on hecho_entrega_mensajeria_diario for column k